# Handwriting Detect 手写识别

Demo1 Homework

Tastror

## 一、开发环境

gpu 相关

- 显卡为 NVIDIA 公司产品，在 帮助 - 系统信息 中查看 GPU 支持的最高 CUDA 版本为 11.7。
- 在 pytorch 官网（2023/3/21）中支持的 CUDA 版本为 11.7 或 11.8

pytorch 相关

```shell
conda create --name py311_torch python=3.11
conda activate py311_torch
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117  # in 2023/3/21, it is 2.0.0
```

<div align="center">

| pytorch 版本 | Python 版本 | cuDNN | CUDA | Windows 版本 |
| ---- | ---- | ---- | ---- | ---- |
| pytorch-2.0.0 | Any (3.11 is used) | Any (8.8.1 is used) | 11.7 | 11 |

</div>

为了后面的画图方便，我们同时安装 matplotlib。

```shell
pip install matplotlib  # in 2023/3/21, it is 3.7.1
```

以及安装 Demo1 中会用到的库

```shell
pip install scipy  # in 2023/3/21, it is 1.10.1
pip install opencv-contrib-python  # in 2023/3/21, it is 4.7.0.72
pip install kornia  # in 2023/3/21, it is 0.6.10
```

接着查看一下 GPU 是否可用

In [1]:
import torch
print(torch.cuda.is_available())

True


可用，说明环境搭建成功，并且尝试运行 Demo1 中的代码，没有问题。

## 二、分析与模型搭建

首先直接运行，看看结果。

运行的最后几行输出如图。

```plaintext
Start Train
Epoch 300/300:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 92/93 [00:00<00:00, 259.91it/s, acc=1, loss=3.25e-7]Start test
Epoch 300/300: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 93/93 [00:00<00:00, 260.92it/s, acc=1, loss=3.25e-7] 
Epoch 300/300: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 497.29it/s, acc=0.726]
```

训练集上的准确率为 100%，而测试集上的数据只有 72.6%，很明显过拟合了。

我们从源数据和模型两个方面进行分析。

### （一）源数据

对于 Demo1 数据集，其有两部分组成：train 和 test，使用 scipy 读取后可用看到其中的具体数据长度。

```python
self.data = scio.loadmat(path)
self.data = self.data['train'] if train else self.data['test']
self.shape = np.shape(self.data)
self.data_shape = self.shape[:2]
self.image_shape = self.shape[2:]
self.length = self.data_shape[0] * self.data_shape[1]

print('length of dataset is', self.length, end=", ")
print('shape of dataset is', self.data_shape, end=", ")
print('shape of image is', self.image_shape)
```

输出为

```plaintext
length of dataset is 3000, shape of dataset is (200, 15), shape of image is (28, 28)  # training dataset
length of dataset is 1000, shape of dataset is (200, 5), shape of image is (28, 28)  # test dataset
```

也就是训练集有 200 个标签，每个标签下有 15 张图片。图片的大小为 28 * 28，和手写数字类似。

很明显，这个图片的数据量是很小的（MNIST 的训练集有 60,000 张，也就是每个标签下有 6000 张图片），所以在训练之前，我们需要进行一下数据增强。

In [ ]:
import numpy as np
from PIL import Image

augment_type_num = 7

def augment(image, augmentation_type, rand_source):

    if augmentation_type == 0:
        pass
    elif augmentation_type == 1:
        # 缩放并平移
        image = Image.fromarray(image)
        image = image.resize((24, 24), resample=Image.BILINEAR)
        image = np.array(image)
        image = np.pad(image, (2, 2), 'constant', constant_values=0)
        if rand_source % 4 == 0:
            image = np.roll(image, 5, axis=0)
        elif rand_source % 4 == 1:
            image = np.roll(image, -5, axis=0)
        elif rand_source % 4 == 2:
            image = np.roll(image, 5, axis=1)
        else:
            image = np.roll(image, -5, axis=1)
    elif augmentation_type == 2:
        # 缩放并平移
        image = Image.fromarray(image)
        image = image.resize((24, 24), resample=Image.BILINEAR)
        image = np.array(image)
        image = np.pad(image, (2, 2), 'constant', constant_values=0)
        if rand_source % 4 == 0:
            image = np.roll(image, -5, axis=0)
            image = np.roll(image, -5, axis=1)
        elif rand_source % 4 == 1:
            image = np.roll(image, 5, axis=0)
            image = np.roll(image, 5, axis=1)
        elif rand_source % 4 == 2:
            image = np.roll(image, 5, axis=0)
            image = np.roll(image, -5, axis=1)
        else:
            image = np.roll(image, -5, axis=0)
            image = np.roll(image, 5, axis=1)
    elif augmentation_type == 3:
        # 旋转操作
        image = Image.fromarray(image)
        image = image.rotate(10)
        image = np.array(image)
    elif augmentation_type == 4:
        # 旋转操作
        image = Image.fromarray(image)
        image = image.rotate(-10)
        image = np.array(image)
    elif augmentation_type == 5:
        # 缩放操作
        image = Image.fromarray(image)
        image = image.resize((24, 24), resample=Image.BILINEAR)
        image = np.array(image)
        image = np.pad(image, (2, 2), 'constant', constant_values=0)
    elif augmentation_type == 6:
        # 缩放操作
        image = Image.fromarray(image)
        image = image.resize((32, 32), resample=Image.BILINEAR)
        image = np.array(image)
        image = image[2:30, 2:30]
    
    return image


以此增强后，数据变为原来的 7 倍，虽然还是不太够，但是已能初步缓解数据不足的问题。

### （二）模型

这里我选择添加一层卷积层，并且由于测试集和训练集的准确率相差太大，所以我添加了 dropout。

```plaintext
Epoch 49/50, Loss: 0.0190, Acc: 95.7554
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 657/657 [00:03<00:00, 173.10it/s] 
Epoch 50/50, Loss: 0.0368, Acc: 95.8160
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 312.85it/s]
Test Accuracy: 0.8780
```

测试集上的准确率为 87.8%，比原来高。

经过多次测试，目前表现最优的模型能达到测试集上 92% 的准确率。代码如下。

In [ ]:
import torch.nn as nn


# 定义模型
class Net(nn.Module):
    def __init__(self, label_num: int = 200):
        super(Net, self).__init__()

        self.lr = 0.001
        self.weight_decay = 0

        self.conv_features_1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.2),
        )

        self.conv_features_2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.2),
        )

        self.conv_features_3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25),
        )

        self.conv_features_4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25),
        )

        self.classifier = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, label_num),
        )
        
    def forward(self, x):
        x = self.conv_features_1(x)
        x = self.conv_features_2(x)
        x = self.conv_features_3(x)
        x = self.conv_features_4(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


### （三）其他修缮

#### 1：数据要足够多
目前数据增强的内容是够用的。如果欠缺还需进一步增强，如进行一定程度的扭曲拉伸等。  
无数据增强和有数据增强的默认模型，75% -> 83%

#### 2：深度要足够描述特征
最开始是目前采用的是四层卷积+池化；新的模型采用了（卷积+卷积+池化）这样的结构一共三层，然后是三个线性层。  
模型 demo-3 和 demo-4 的区别，91% -> 92.5%

#### 2：Dropout 的位置对模型的收敛速度影响巨大
一种是 Conv2D -> Dropout -> BatchNorm -> ReLU；  
另一种是 Conv2D -> BatchNorm -> ReLU -> Dropout。  
这两种写法之间的主要区别在于 dropout（丢弃）和 Batch Normalization（批归一化）的顺序。  
在第一种写法中，dropout 被应用于 Batch Normalization 前。因此，在使用 dropout 之后，特征图的方差和均值相比原来会产生不稳定的变化，并且 dropout 可能会丢弃对某些特征的有用贡献。  
在第二种写法中，dropout 被放在 ReLU 激活函数之后，这意味着它被应用于激活后的特征图。在这种情况下，dropout 不会影响特征图的方差和均值，不容易丢弃对模型学习有用的信息。  
因此，第二种写法通常更受欢迎，因为它可以更好地防止过拟合，而且对模型的训练和性能更有益。  
无数据增强的 150 epoch 速训，模型 demo-3，第一种为 77%（主要是因为增长太慢），第二种为 87%

#### 4：降低学习率
学习率从 0.001 降低到了 0.0005，之前后期无法进步的学习就能够进行了。  
最终我们降为了 0.0002
```python
# Adam
self.lr = 0.0002
self.weight_decay = 0.005
``` 
模型 demo-4，400 epoch 内，92% -> 93%

#### 5：使用 weight_decay 正则（或者使用 AdamW，自带 weight_decay）
默认 Adam 的 weight_decay 是 0。若要进行 L2 正则化，可手动赋值为 0.001 等值。AdamW 默认参数值是 0.01。

#### 6：重复数据
对于数据中重复或类似字符相关的内容，在一一比对后，统计出它们一共有 5 组（4 组两两相似和 1 组 3 个相似）。  
其准确率上限的平均情况为 97%。在去除后可以提升这 3% 的上限。

```python
redundant = [
    (0, 81), (27, 39), (45, 195), (99, 114), (120, 121, 122)
]
```

以上内容均为相似（前四个）或重复（最后一个）内容，将只保留第一个标签，其余映射到末尾（删除也可，这里采取的是映射，81->194, 39->196, 114->197, 121->198, 122->199）。


最终代码如下。

In [ ]:
import torch.nn as nn


# 定义模型
class Net(nn.Module):
    def __init__(self, label_num: int = 200):
        super(Net, self).__init__()

        # Adam
        self.lr = 0.0002
        self.weight_decay = 0.005

        self.conv_features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),

            nn.MaxPool2d(kernel_size=2),

            # nn.Upsample(scale_factor=2, mode='bilinear'),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),

            nn.MaxPool2d(kernel_size=2),

            nn.Upsample(scale_factor=2, mode='bilinear'),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),

            nn.MaxPool2d(kernel_size=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(3136, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),

            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),

            nn.Linear(512, label_num),
            nn.Dropout(0.2),
        )
        
    def forward(self, x):
        x = self.conv_features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


在不同数据集下的测试结果为（epoch <= 400）

| 数据集 | 准确率 |
| --- | --- |
| 训练集 | 99.69% |
| 增强训练集（训练时使用） | 98.93% |
| 测试集 | 95.05% |
| 增强测试集 | 92.16% |

 ![inference gif](https://s2.loli.net/2023/03/31/ptNwU158ZIRarQe.png)  
infer.py 展示
